# Workshop 5 - A random forest pixel classifier

This week we had a quick look at the usage of ML algorithms in image analysis.

The lectures gave a very broad and general overview of different ML algorithms.

In this workshop we will take a much more practical approach and build a pixel classifier using Random Forests (RF).

## Introduction

### Random Forests

Random Forests is a supervised learning algorithm that works by generating a large number of classification trees, that work on a bootstrapped sample of the training observations and of the training features.

For the context of this workshop, you don't need to exactly know how RF works, but if you want to refresh your mind, I have attached the slides from the IBMS3 lecture you should have all had last year.

### Aim of the task

For this workshop we want to train a pixel classifier that can classify pixels in an image into a certain number of classes.

We will provide one image and label just a few pixels of different classes, and hopefully RF will work its magic and classify the rest! We will then try to classify new images.

We will try a few different things and try to see how good our classifier is, by comparing our results with ground truth segmentation. 

## The data

I have included 4 images of DAPI-stained nuclei (source: Florian Kromp et al. 2020, ["An annotated fluorescence image dataset for training nuclear segmentation methods"](https://www.nature.com/articles/s41597-020-00608-w), CC0) and corresponding ground truth labels. Note that, while the ground truth labels show instance segmentations, we will only do a semantic segmentation here. We will therefore convert the ground truth labels to a binary mask.

Let's start by plotting the images and their GT labels.

In [ ]:
# Import libraries
import matplotlib.pyplot as plt
import numpy as np
import os

# The names of the folders where our files are stored
# The file naming is the same in the two folders
GT_folder = "GT"
img_folder = "Images"

# Read the filenames of the images
filenames = os.listdir(img_folder)
print(filenames)

# Loop through the files, read the images and GT data (remember to binarise it!) and plot them
# Your code here

We will now just mark a few pixels as belonging to a nucleus or the background.
We could manually draw a mask but for the sake of simplicity we'll just select a crop of the image.

Let's find a 200x200 square in the image containing both nuclei and background.
We will start working on the first image (`Normal1.tif`).

In reality, you might do something like this:

![Example of marking a few pixels](Normal1_marked_pixels.jpg)

Here I marked a few background pixels in red and a few nucleus pixels in blue.

For simplicity, here we are going to use a cropped version of the image, but you could try and save a mask with those strokes and fetch the intensity and labels of the non zero pixels to use as training data.

In [ ]:
# Read the first image and the GT (make sure to binarise it!)
# img = _____
# gt = _____

# The coordinates of our training region
# We can just grab GT labels, but in a real-world situation we would probably have to 
# manually "paint" over the image to set the labels 
x, y, width, height = (500, 500, 200, 200)
training = img[y:y+height, x:x+width]

# Plot the image, the cropped area and the GT labels
# Your code here

The input features need to have observations as rows and features in columns. We only have one feature here (intensity) and 100x100=10000 observations. We can therefore reshape our training data into a single column.

The training labels can be reshaped into a single row.

Remember that this 200x200 square is the only data that our classifier will see during training.

In [3]:
x_train = training.reshape(-1, 1) # -1 means the number of rows will be determined automatically
y_train = gt[y:y+height, x:x+width].ravel()

## Training the classifier

We can finally create a Random Forest classifier.

Import `RandomForestClassifier` from the `sklearn.ensemble` module and initialise it with some parameters of your choice. The number of trees is passed through the `n_estimators` parameter, and you can set the maximum depth of the trees with the `max_depth` parameter.

In [4]:
# Hyperparameters
n_estimators = ___
max_depth = ___

# Initialise the classifier with the hyperparameters of our choice. 
# You can set the random state for reproducibility
rf = RandomForestClassifier(_____)

# Now train the classifier with the pixels of the nuclei and the background
rf = rf.fit(x_train, y_train)

Now that the classifier is trained, we can use it to classify all the pixels in the image! Again, we reshape the image into a single column, then reshape back the results into a 2D image. 

In [5]:
## Prediction 
label_pred = rf.predict(img.ravel().reshape(-1,1))
label_pred = label_pred.reshape(img.shape)

In [ ]:
# Now, print the results and compare with GT!

# Your code here

This should be pretty good! Even just by using a single feature (the intensity) the classifier does a good job. This is because the two classes are clearly separable by the intensity. 

The classifier is not perfect, though, failing especially at the edges of cells. 
Let's see how this performs on the other images

In [1]:
# Loop through the files and predict the labels with the classifier.
# Plot the results.
# Your code here

**Do you see anything worth noting?**

## Accuracy metrics

Before we move onto improving our classifier, we need a way to assess the goodness of our model.

### Pixel accuracy 

A simple solution is to count the percentage of correctly classified pixels. This is called **pixel accuracy** and is defined as:

$\text{Pixel accuracy} = \Large \frac{\text{correctly classified pixels}}{\text{total pixels}}$

This is a very simple metric, but it can be **extremely misleading**. Say that 95% of your image is background and your classifier predicts that every single pixel is background... you still end up with 95% accuracy, which is obviously not great in this case!

*Can we do better?*

Two commonly used metrics for evaluating segmentation are the **IoU** (intersection over union, or Jaccard coefficient) and the **F1 score** (or Dice coefficient).

### IoU

**IoU** is defined as:

$IoU = \Large \frac{\text{intersection of pixels}}{\text{union of pixels}}$

This is calculated for each class and then averaged.

The *intersection* is the number of pixels that are classified in the same way (e.g. background) in both GT and prediction.
The *union* is the total number of non-overlapping pixels, classified in some way (e.g. background) in GT or prediction.


### F1 score

The F1 score is defined as:

$F1 = \Large \frac{2 \times \text{intersection of pixels}}{\text{union of pixels} + \text{intersection of pixels}}$


Both the Iou and the F1 score range from 0 to 1; they are very similar, and always correlated. However, they might give slightly different results depending on the problem (see [this discussion on the topic](https://stats.stackexchange.com/questions/273537/f1-dice-score-vs-iou/276144)).  


So, let's define three functions to calculate these metrics

In [2]:
def pixel_accuracy(y_true, y_pred):
    """Returns pixel accuracy of the predicted labels

    Args:
        y_true (np.array): The ground truth labels
        y_pred (np.array): The predicted labels

    Returns:
        float: The pixel accuracy, ranging from 0 to 1
    """
    # Ensure inputs are the same shape
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

    # Calculate the number of pixels that are correctly labelled
    return np.sum(y_true == y_pred) / len(y_true)

def IoU(y_true, y_pred):
    """
    Returns the Intersection over Union (IoU) of the predicted labels
    
        Args:
            y_true (np.array): The ground truth labels
            y_pred (np.array): The predicted labels
    
        Returns:
            float: The IoU, ranging from 0 to 1
        """

    # Ensure inputs are the same shape
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    iou_score = np.sum(intersection) / np.sum(union)

    return iou_score

def f1_score(y_true, y_pred):
    """
    Returns the F1 score of the predicted labels

        Args:
            y_true (np.array): The ground truth labels
            y_pred (np.array): The predicted labels
    
        Returns:
            float: The F1 score, ranging from 0 to 1
        """

    # Ensure inputs are the same shape
    assert y_true.shape == y_pred.shape, "y_true and y_pred must have the same shape"

    intersection = np.logical_and(y_true, y_pred)
    union = np.logical_or(y_true, y_pred)
    
    F1 = (2 * np.sum(intersection)) / (np.sum(union) + np.sum(intersection))

    return F1

In [ ]:
# Now loop through the files and calculate the accuracy, IoU, 
# and F1 score for each image
# Store the results of the predictions in arrays, so that you can plot it

Think about the results you just got. Are they in line with what you observed by looking at the prediction results? What do the segmentation accuracy plots tell you? 

## Extra exercises

1. Try to add new training data from another image. Does that improve the results? 

We only used a single feature (intensity) to train the classifier. This is OK here because intensity discriminates well nuclei and background, however, you could improve classification by passing more features to the classifier. For example you could add the gradient magnitude, or the entropy, or GLCM features! You could also calculate those features on a series of gaussian-filtered versions of the image (with different $\sigma$), to get information from neighbouring pixels as well. 

2. Try segmenting the `skin.jpg` image similarly to what we have done above! I am not providing GT for this image, but you can evaluate the results visually. 

And that's it! Well done, you made it to the end of the workshop 5! I hope you enjoyed it; if you have any questions, please [ask](mailto:nicola.romano@ed.ac.uk)!